In [4]:
from gensim.models import Word2Vec
import re
import matplotlib.pyplot as plt
import os
import numpy as np
from nltk.corpus import stopwords
from multiprocessing import cpu_count

C:\Users\llair\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
def read_file(filename):
    stopWords = set(stopwords.words('english'))
    with open(filename, 'r',encoding = 'utf-8', errors = 'surrogateescape') as f:
        doc = f.read()
        reviews = doc.split('<EOR>')
    data = [];
    for r in reviews[:-1]:
        temp = re.sub(r'([^\s\w]|_)+', '', r)
        temp = temp.split()
        temp = list(filter(lambda a: a != '', temp))
        temp2 = []
        for t in temp:
            t = t.lower()
            if t not in stopWords:
                temp2.append(t)
        data.append(temp2)
    rating = reviews[len(reviews)-1]
    rating = rating[10:13]
    return(data,rating)

def read_directory(directory, verbose = False):
    data = []
    id_list = []
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            if(verbose):
                print("Starting file: "+filename)
            business_id = filename.split('.txt')[0]
            filename = directory + '/' + filename
            temp = read_file(filename)
            if(verbose):
                print("Finished file: "+business_id+".txt")
            for t in temp: 
                data.append(t)
                id_list.append(business_id)
    return((data,id_list))

In [19]:
def generator_read_dir(directory):
    files = os.listdir(directory)
    i = 0
    n = len(files)
    while i < n:
        filename = files[i]
        i = i+1
        if filename.endswith('.txt'):
            business_id = filename.split('.txt')[0]
            filename = directory + '/' + filename
            temp = read_file(filename)
            yield(temp,business_id)

AttributeError: 'list' object has no attribute 'size'

In [13]:
def read_reviews(directory):
    files = os.listdir(directory)
    i = 0
    n = len(files)
    while i < n:
        filename = files[i]
        i = i+1
        if filename.endswith('.txt'):
            filename = directory + '/' + filename
            temp = read_file(filename)
            yield(temp[0])
directory = "reviews_3mo"
data = read_reviews(directory) #We only want the review text
d = []
for i in data:
    for j in i:
        d.append(j)
num_cores = cpu_count()
model = Word2Vec(d, size = 100, window = 5, min_count = 1, workers = num_cores)
model.save('word2vec_test.model')
model.wv.save('wordvecs_test.kv') #Save keyed vectors as well

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
#Basic word2vec example
num_cores = cpu_count()
model = Word2Vec(data, size = 100, window = 5, min_count = 1, workers = num_cores)
model.save('word2vec_test.model')
word = 'hello'
vector = model.wv[word] #Returns numpy array of vector embedding of word

In [ ]:
#Loading word2vec model from model file
model = Word2Vec.load("word2vec_test.model")
#If we have new data to train with
#Params: 
#total_examples = number of sentences
#epochs = number of iterations
model.train(data, total_examples = 1, epochs = 1)

In [ ]:
#KeyedVectors allow us to ditch the neural net in favor of just vectors
model.wv.save('wordvecs.kv')
wordvecs = KeyedVectors.load("wordvecs.kv", mmap = 'r')
word = 'hello'
vector = wordvecs[word]
#We can also ditch the model in this way
wordvecs = model.wv #wordvecs will be the keyedvectors
del model

In [ ]:
#Interesting functionality built into gensim
#Positive words are words we want to be similar to
#Negative words are words we want to be different to
#most_similar will find the word which is closest to P1+P2+...+Pn-N1-N2-...-Nn
result = word_vectors.most_similar(positive = [P1,P2], negative = [N1])
#Or we can pass in a vector directly(Need the whole model though)
result = model.most_similar(positive = [vector], topn = i)
#we can check which of a given set is closest to a word
result = word_vectors.most_similar_to_given(word1, wordlist)